In [17]:
#!pip install scikit-learn
#
#!pip install feast 



from typing import NamedTuple
import pandas as pd 
import pickle
import tensorflow as tf
import numpy as np 

from kfp import dsl

from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import time
import numpy as np


In [18]:
#!feast init
!cd viable_cat
!feast apply 

/usr/bin/sh: 1: feast: not found


In [19]:
pip install kubeflow-kale

  Using cached nbformat-4.4.0-py2.py3-none-any.whl (155 kB)
  Using cached Jinja2-2.11.3-py2.py3-none-any.whl (125 kB)


  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.4
    Uninstalling Jinja2-3.1.4:
      Successfully uninstalled Jinja2-3.1.4
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.10.4
    Uninstalling nbformat-5.10.4:
      Successfully uninstalled nbformat-5.10.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.2.1 requires jinja2>=3.0, but you have jinja2 2.11.3 which is incompatible.
nbconvert 7.2.1 requires nbformat>=5.1, but you have nbformat 4.4.0 which is incompatible.
nbclient 0.7.0 requires nbformat>=5.0, but you have nbformat 4.4.0 which is incompatible.
jupyterlab 3.4.4 requires jupyterlab-server~=2.10, but you have jupyterlab-server 1.2.0 which is incompatible.
jupyter-server 1.21.0 requires nbformat>=5.2.0, but you have nbformat 4.4.0 which is incompatible.

[notice] A new release of pip 

In [20]:
!pip install "jupyterlab>=2.0.0,<3.0.0"

# install the extension
!jupyter labextension install kubeflow-kale-labextension

  Using cached jupyterlab-2.3.2-py3-none-any.whl (7.9 MB)
  Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)


  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: nbformat
    Found existing installation: nbformat 4.4.0
    Uninstalling nbformat-4.4.0:
      Successfully uninstalled nbformat-4.4.0
  Attempting uninstall: jupyterlab
    Found existing installation: jupyterlab 3.4.4
    Uninstalling jupyterlab-3.4.4:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/usr/local/etc/jupyter/jupyter_notebook_config.d/jupyterlab.json'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch


In [21]:

!pip install kfp




[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
#test 
def read_data() -> list:
    data = [1, 2, 3, 4, 5]
    return data



import numpy as np 
#import kfp.dsl

from typing import NamedTuple
from minio import Minio

#NamedTuple('Outputs', [('X_train', str), ('X_test', str), ('y_train', str), ('y_test', str)]):
def read_file() -> None:
    
    def zscore_normalization(df, name):
        mean = df[name].mean()
        sd = df[name].std()
        df[name] = (df[name] - mean) / sd

    def encode_text(df, name):
        from sklearn.preprocessing import OrdinalEncoder
        enc = OrdinalEncoder()
        data = enc.fit_transform(df[name].values.reshape(-1,1))
        df[name] = data.flatten()

    def column_names():
        f = open("kddcup.names.txt")
        s = f.read()
        arr = s.split("\n")[1:-1]
        cols = [a[0:a.index(":")] for a in arr]
        cols.append("outcome")
        return cols

    def preprocess(df):
        #df.columns = column_names()
        df.columns = [
        'duration',
        'protocol_type',
        'service',
        'flag',
        'src_bytes',
        'dst_bytes',
        'land',
        'wrong_fragment',
        'urgent',
        'hot',
        'num_failed_logins',
        'logged_in',
        'num_compromised',
        'root_shell',
        'su_attempted',
        'num_root',
        'num_file_creations',
        'num_shells',
        'num_access_files',
        'num_outbound_cmds',
        'is_host_login',
        'is_guest_login',
        'count',
        'srv_count',
        'serror_rate',
        'srv_serror_rate',
        'rerror_rate',
        'srv_rerror_rate',
        'same_srv_rate',
        'diff_srv_rate',
        'srv_diff_host_rate',
        'dst_host_count',
        'dst_host_srv_count',
        'dst_host_same_srv_rate',
        'dst_host_diff_srv_rate',
        'dst_host_same_src_port_rate',
        'dst_host_srv_diff_host_rate',
        'dst_host_serror_rate',
        'dst_host_srv_serror_rate',
        'dst_host_rerror_rate',
        'dst_host_srv_rerror_rate',
        'outcome'
        ]
        for col in df.columns:
            t = (df[col].dtype)
            if t == int or t == float:
                zscore_normalization(df, col)
            else:
                encode_text(df, col)
        df.dropna(inplace=True,axis=1)
        for col in df.columns:
            if len(df[col].unique()) == 1:
                df.drop(col, inplace=True,axis=1)

        df.loc[df['outcome'] != "normal.", 'outcome']  = 1
        df.loc[df['outcome'] == "normal.", 'outcome']  = 0

        correlation = df.corrwith(df['outcome'])
        for num in correlation:
            if num >= -0.05 and num <= 0.05:
                df.drop(df.columns[row], axis=1, inplace=True)
        return df
    
    
    from sklearn.model_selection import train_test_split
    import os 
    import pandas as pd
    import numpy as np 
    from sklearn import metrics
    
  
    
    file_path = 'https://raw.githubusercontent.com/tsimhadri-ews/internproject/intrusion-detection-0/src/kddcup.data_10_percent_corrected.csv'
    df = pd.read_csv(file_path)
    df = preprocess(df)
    
    #minio-serve.kubeflow:127.0.0.1:9000
    minio_client = Minio(
        "192.168.61.216:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )
    minio_bucket = "mlpipeline"
    

    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["outcome"]), df["outcome"], test_size=0.2)
    """
    
    

    #metrics.log_metric("Len x_train", X_train.shape[0])
    #metrics.log_metric("Len x_test", X_test.shape[0])

    
    directory = '/home/jovyan/tmp/data'

    # Ensure the directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

     # Save the data
    np.save(os.path.join(directory, 'X_train.npy'), X_train)
    np.save(os.path.join(directory, 'X_test.npy'), X_test)
    np.save(os.path.join(directory, 'y_train.npy'), y_train)
    np.save(os.path.join(directory, 'y_test.npy'), y_test)
    

    print("Data saved")
    print("X_train:", X_train)
    print("X_test:", X_test)
    print("y_train:", y_train)
    print("y_test:", y_test)

    return [os.path.join(directory, 'X_train.npy'),
            os.path.join(directory, 'X_test.npy'),
            os.path.join(directory, 'y_train.npy'),
            os.path.join(directory, 'y_test.npy')]
    
    print("test and train data")
    print(X_train, X_test, y_train, y_test)
    return (X_train, X_test, y_train, y_test)
    """
    
    np.save("tmp/x_train.npy",X_train)
    minio_client.fput_object(minio_bucket,"x_train","tmp/x_train.npy")

    np.save("tmp/y_train.npy",y_train)
    minio_client.fput_object(minio_bucket,"y_train","tmp/y_train.npy")

    np.save("tmp/x_test.npy",X_test)
    minio_client.fput_object(minio_bucket,"x_test","tmp/x_test.npy")

    np.save("tmp/y_test.npy",y_test)
    
    minio_client.fput_object(minio_bucket,"y_test","tmp/y_test.npy")
   
    print(minio_client)
    
    print(f"x_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    
    

    print(f"x_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")

read_file()



/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


x_train shape: (395216, 39)
y_train shape: (395216,)
x_test shape: (98804, 39)
y_test shape: (98804,)


In [23]:
import pandas as pd


def train_op(X_train_path: str, X_test_path: str, y_train_path: str, y_test_path: str) -> None:
    import pickle
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, f1_score
    import numpy as np 
    import json
    import os 
    import tarfile 
    
    # Convert JSON strings back to DataFrames

    #saved the model to dir 
    #print(X_train)
    
    new_path = '/home/jovyan/tmp/data/X_test'
    data_file = "train_data.csv"
    
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    
    with tarfile.open(X_test_path, "r:gz") as tar:
        tar.extractall(path=new_path)
        print("Extraction complete.")

    # Load the data from the extracted file
    data_path = open(new_path)
   
    X_train = np.load(data_path)
    print(X_train)
    """
    X_test = np.load(X_test_path)
    y_train = np.load(y_train_path)
    y_test = np.load(y_test_path)
    """
    
    
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    
    #Kubeflox metrics export
    
    #metrics.log_metric("Test accuracy", accuracy)
    #metrics.log_metric("F1 Score", f1)
    
    #implement TFServing 

    with open(f'model.pkl', 'wb') as f:
        pickle.dump(model, f)


In [24]:
import pickle
import pandas as pd
import numpy as np

def predict_op(input_data:str) -> str:
    with open('model.pkl', 'rb') as f:
        model = pickle.load(f)
    return str(model.predict(test_data))

In [25]:
from kfp import components


#preprocess_op = components.func_to_container_op(func=preprocess_op, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn'])

read_csv_op = components.func_to_container_op(func=read_file, output_component_file='read.yaml', base_image='python:3.7', packages_to_install=['pandas','scikit-learn', 'kfp', 'numpy'])
#preprocess_op = components.func_to_container_op(func=preprocess_op, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn'])
#preprocess_op = components.func_to_container_op(func=preprocess_op, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn'])
#train_op = components.func_to_container_op(func=train_op, output_component_file='train.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn','numpy' ])
#predict_op = components.func_to_container_op(func=predict_op,output_component_file='predict.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn', 'numpy'])



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_576/1010570788.py", line 6, in <module>
    read_csv_op = components.func_to_container_op(func=read_file, output_component_file='read.yaml', base_image='python:3.7', packages_to_install=['pandas','scikit-learn', 'kfp', 'numpy'])
  File "/usr/local/lib/python3.9/site-packages/kfp/components/_python_op.py", line 946, in func_to_container_op
  File "/usr/local/lib/python3.9/site-packages/kfp/components/_components.py", line 428, in _create_task_factory_from_component_spec
  File "/usr/local/lib/python3.9/site-packages/kfp/components/_dynamic.py", line 47, in create_function_from_parameters
    modified_code = types.CodeType(
TypeError: code() takes at least 14 arguments (13 given)

During handling of the above exception, another exception occurred:

Traceback (most recent cal

In [2]:
import kfp
from kfp import dsl

read_data_op = kfp.components.load_component_from_file('read.yaml')
#preprocess_data_op = kfp.components.load_component_from_file('preprocess.yaml')
#train_op = kfp.components.load_component_from_file('train.yaml')
#predict_op = kfp.components.load_component_from_file('predict.yaml')


@dsl.pipeline(
    name='Machine Learning Pipeline',
    description='A pipeline to preprocess, train, and predict using sklearn and tensorflow'
)




def ml_pipeline():  #(file_path:str)
    #read_csv = read_csv_op()
    preprocess = read_csv_op()
    #print(preprocess.outputs['X_test'])
    print(preprocess.outputs)
    print(preprocess.outputs['X_train'])
    
    """
    train = train_op(preprocess.outputs['X_train'], 
                     preprocess.outputs['X_test'], 
                     preprocess.outputs['y_train'], 
                     preprocess.outputs['y_test']).after(preprocess)
    
    """
    
    #predict_task = predict_op(preprocess.outputs['X_test'])

    
    
# Compile the pipeline
kfp.compiler.Compiler().compile(ml_pipeline, 'pipeline46.yaml')

    

TypeError: code() takes at least 14 arguments (13 given)

In [ ]:
from kfp import compiler

pipeline_func = ml_pipeline
pipeline_filename = pipeline_func.__name__ + '.yaml'

compiler.Compiler().compile(pipeline_func, pipeline_filename)